From https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools

In [1]:
import pandas as pd
df = pd.read_csv("salted-tahini-chocolate-chip-cookies.csv")
tahini = df.copy()

In [2]:
tahini.head()

,userDisplayName,commentBody
0,lmk,Yum. These took much longer than 16 minutes t...
1,Sonya,If you follow the recipe as written the tahini...
2,KV,I have made these cookies 5 times. My advice i...
3,MaryN,I liked this- the tahini is slightly more subt...
4,Maggie B,Used Shaila M's tweaks. Baked first tray strai...


In [3]:
import numpy as np
import nltk

In [4]:
from nltk.corpus import stopwords

In [5]:
import gensim

In [6]:
from nltk.stem import WordNetLemmatizer#, PorterStemmer

In [7]:
from nltk.tokenize import word_tokenize

In [8]:
## https://stackoverflow.com/questions/66759852/no-module-named-pyldavis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

#import pyLDAvis
# import pyLDAvis.gensim

In [9]:
def get_lda_objects(text):
    nltk.download('stopwords')    
    stop=set(stopwords.words('english'))

    
    def _preprocess_text(text):
        corpus=[]
#         stem=PorterStemmer()
        lem=WordNetLemmatizer()
        for comment in text:
            words=[w for w in word_tokenize(comment) if (w not in stop)]

            words=[lem.lemmatize(w) for w in words if len(w)>2]

            corpus.append(words)
        return corpus
    
    corpus=_preprocess_text(text)
    
    dic=gensim.corpora.Dictionary(corpus)
    bow_corpus = [dic.doc2bow(doc) for doc in corpus]
    
    lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 4, 
                                   id2word = dic,                                    
                                   passes = 10,
                                   workers = 2)
    
    return lda_model, bow_corpus, dic

def plot_lda_vis(lda_model, bow_corpus, dic):
    pyLDAvis.enable_notebook()
    vis = gensimvis.prepare(lda_model, bow_corpus, dic)
    return vis

In [10]:
lda_model, bow_corpus, dic = get_lda_objects(tahini['commentBody'])

[nltk_data] Downloading package stopwords to /Users/Nida/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
lda_model.show_topics()

[(0,
  '0.030*"cooky" + 0.013*"recipe" + 0.012*"tahini" + 0.011*"chocolate" + 0.010*"dough" + 0.010*"made" + 0.009*"1/2" + 0.009*"chip" + 0.008*"cookie" + 0.008*"sugar"'),
 (1,
  '0.017*"cooky" + 0.014*"chip" + 0.014*"chocolate" + 0.013*"tahini" + 0.011*"recipe" + 0.010*"dough" + 0.010*"used" + 0.009*"make" + 0.009*"flour" + 0.008*"n\'t"'),
 (2,
  '0.021*"cooky" + 0.020*"tahini" + 0.012*"time" + 0.011*"minute" + 0.010*"used" + 0.010*"cookie" + 0.009*"make" + 0.009*"scoop" + 0.009*"n\'t" + 0.008*"The"'),
 (3,
  '0.016*"recipe" + 0.015*"cooky" + 0.014*"cookie" + 0.013*"half" + 0.013*"chocolate" + 0.012*"made" + 0.011*"sugar" + 0.011*"dough" + 0.010*"chip" + 0.010*"time"')]

In [12]:
plot_lda_vis(lda_model, bow_corpus, dic)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.001120 -0.085069       1        1  32.722671
0     -0.020870  0.023731       2        1  27.696613
2     -0.070388  0.030764       3        1  20.498843
1      0.090138  0.030575       4        1  19.081873, topic_info=       Term        Freq       Total Category  logprob  loglift
41   tahini  105.000000  105.000000  Default  30.0000  30.0000
197    meal   15.000000   15.000000  Default  29.0000  29.0000
209    half   53.000000   53.000000  Default  28.0000  28.0000
83     chip   78.000000   78.000000  Default  27.0000  27.0000
320     1/2   36.000000   36.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
320     1/2    7.175009   36.723802   Topic4  -5.4623   0.0236
327  baking    7.323687   43.729668   Topic4  -5.4418  -0.1305
108    They    6.792155   34.967187   Topic4  -5.5171   0.0178
63     made    7.105825   81.254580   Topic4  -5.4720  -0.7802
230   sugar    6.651838   69.784743   Topic4  -5.5380  -0.6941

[318 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
353       4  0.850743      -used
21        1  0.238057        ...
21        2  0.178543        ...
21        3  0.535629        ...
21        4  0.119029        ...
...     ...       ...        ...
384       4  0.228938  wonderful
43        1  0.465488      would
43        2  0.126951      would
43        3  0.338537      would
43        4  0.084634      would

[507 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

Need to figure out how to interpret this / what this visualization output means... 